# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [92]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [93]:
vacation_data = pd.read_csv("weather_data.csv")
vacation_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,malmesbury,-33.46,18.73,64.40,93,75,5.82,ZA,1607463214
1,cherskiy,68.75,161.30,-12.86,89,100,7.00,RU,1607463214
2,atuona,-9.80,-139.03,78.60,73,1,12.15,PF,1607463214
3,taga,46.95,24.05,34.00,86,20,6.93,RO,1607463214
4,hobart,-42.88,147.33,62.60,48,75,13.87,AU,1607463113
5,saskylakh,71.92,114.08,-16.02,87,97,3.53,RU,1607463214
6,qaanaaq,77.48,-69.36,-1.64,80,52,3.24,GL,1607463214
7,arlit,18.74,7.39,78.44,19,0,8.23,NE,1607463215
8,kapaa,22.08,-159.32,82.40,69,20,13.87,US,1607463215
9,bluff,-46.60,168.33,60.01,73,96,10.00,NZ,1607463215


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [94]:
gmaps.configure(api_key=g_key)

locations = vacation_data[["Lat", "Lng"]].astype(float)
humidity = vacation_data["Humidity"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [95]:
new_vacation = vacation_data.loc[(vacation_data["Max Temp"] < 80) & (vacation_data["Max Temp"] > 70) & (vacation_data["Wind Speed"] < 10) 
                                 & (vacation_data["Cloudiness"] == 0)] 
new_vacation.dropna().head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,arlit,18.74,7.39,78.44,19,0,8.23,NE,1607463215


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [96]:
hotel_df = new_vacation.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
7,arlit,NE,18.74,7.39,


In [97]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"keyword": "hotel", "type": "hotel", "radius": 5000, "key": g_key}

#iterate through hotel dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f'{lat}, {lng}'
    
    #assemble URL and API request
    print(f'Retrieving results for {index}: {city_name}.')
    response = requests.get(url, params=params).json()
    
    results = response["results"]
    
    try:
        print(f'The closest hotel in {city_name} is {results[0]["name"]}.')
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
          
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")   
    
    time.sleep(1)
          
print("----------------------------")                               

Retrieving results for 7: arlit.
The closest hotel in arlit is Hôtel Telwa Bungalow.
----------------------------


In [98]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
7,arlit,NE,18.74,7.39,Hôtel Telwa Bungalow


In [99]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [100]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))